In [458]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist

from hmmlearn import hmm
from pgmpy.models import DiscreteBayesianNetwork
from pyro.distributions.hmm import DiscreteHMM
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

# Ad Data Preprocessing

In [460]:
# Define column renaming mapping (adjust if your CSV headers differ slightly)
rename_map = {
        'Device platform': 'Device_platform',
        'Platform': 'Platform',
        'Custom Audience Defined': 'Use_Custom_Audience',
        'Exclusion Defined': 'Use_Exclusions',
        'Amount spent': 'Amount_Spent',
        'CPM (cost per 1,000 impressions)': 'CPM',
        'Clicks (all)': 'Clicks',
        'Leads': 'Number_of_Leads',
        'Cost per Lead': 'CPL',
        'Objective': 'Campaign_Objective',
        'Headline_Local': 'Headline_Local',
        'Headline_Event': 'Headline_Event',
        'Headline_Exclusivity': 'Headline_Exclusivity',
        'Headline_Rental': 'Headline_Rental',
        'Headline_Returns': 'Headline_Returns',
        'Headline_Discounts': 'Headline_Discounts'
}

# Define all columns to keep based on rename_map keys + impressions/clicks
cols_to_keep_original = list(rename_map.keys()) + ['Month', 'Impressions', 'Reach', 'Frequency'] # Clicks is already in rename_map keys

df = pd.read_csv('./Data/data.csv')
# Select and rename
data = df[cols_to_keep_original].rename(columns=rename_map)

In [461]:
#Calculate CTR

data['Clicks'].fillna(0)
data['Impressions'].fillna(0)
data['Clicks'] = data['Clicks'].astype(int)
data['Impressions'] = data['Impressions'].astype(int)
data['CTR'] = data['Clicks'] / data['Impressions']*100
data['CTR'] = data['CTR'].fillna(0)


#Convert Y/N to 1/0
cols_to_convert = ['Use_Custom_Audience', 'Use_Exclusions']
data[cols_to_convert] = data[cols_to_convert].replace({'Y': 1, 'N': 0})

# Set CPL to 100000 if lead count is 0
condition = (data['Number_of_Leads'] == 0) | pd.isna(data['Number_of_Leads'])
data['CPL'] = np.where(condition, 100000, data['Amount_Spent'] / data['Number_of_Leads'])

# Keep only Facebook or Instagram
data = data[data['Platform'].isin(['facebook', 'instagram'])]


#data.head

C:\Users\Victor\AppData\Local\Temp\ipykernel_4732\952254001.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[cols_to_convert] = data[cols_to_convert].replace({'Y': 1, 'N': 0})


In [462]:
# 1. CPM (Percentiles)
# Let's use 5 bins (quintiles: 0-20%, 20-40%, 40-60%, 60-80%, 80-100%).

num_cpm_bins = 5
cpm_labels = [f'Quantile {i+1}' for i in range(num_cpm_bins)] # e.g., ['Quantile 1', 'Quantile 2', ...]
# Alternative descriptive labels:
# cpm_labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

data['CPM_Category'] = pd.qcut(data['CPM'],
                                q=num_cpm_bins,
                                labels=cpm_labels,
                                duplicates='drop') # Important if many values are the same
data['CPM_Category'] = data['CPM_Category'].cat.add_categories(['No spend'])
condition = (data['Amount_Spent'] == 0) | (data['Amount_Spent'].isna())
data.loc[condition, 'CPM_Category'] = 'No spend'

    
    
# 2. Amount Spent (Buckets of 100s)
# Bins: [0, 100), [100, 200), [200, 300), ...
max_spent = data['Amount_Spent'].max()
# Create bin edges from 0 up to slightly above the max, stepping by 100
amount_bins = np.arange(0, max_spent + 100, 100)
# Create labels like '0-100', '100-200', ...
amount_labels = [f'{int(amount_bins[i])}-{int(amount_bins[i+1])}' for i in range(len(amount_bins)-1)]
data['Amount_Spent_Category'] = pd.cut(data['Amount_Spent'],
                                       bins=amount_bins,
                                       labels=amount_labels,
                                       right=False,
                                       include_lowest=True)
data['Amount_Spent_Category'] = data['Amount_Spent_Category'].cat.add_categories(['No spend'])
condition = (data['Amount_Spent'] == 0) | (data['Amount_Spent'].isna())
data.loc[condition, 'Amount_Spent_Category'] = 'No spend'

print("Amount_Spent binned into 100s.")


# 3. CTR (Buckets of 1%)
# Bins: [0, 1), [1, 2), [2, 3), ...
max_ctr = data['CTR'].max()
ctr_bins = np.arange(0, max_ctr + 1, 1)
ctr_labels = [f'{int(ctr_bins[i])}%-{int(ctr_bins[i+1])}%' for i in range(len(ctr_bins)-1)]
data['CTR_Category'] = pd.cut(data['CTR'],
                              bins=ctr_bins,
                              labels=ctr_labels,
                              right=False,
                              include_lowest=True)
data['CTR_Category'] = data['CTR_Category'].cat.add_categories(['No clicks'])
data['CTR_Category'] = data['CTR_Category'].cat.add_categories(['Invalid CTR'])

condition = (data['CTR'] == 0) | (data['CTR'].isna() | (data['CTR'].isnull()) )
data.loc[condition, 'CTR_Category'] = 'No clicks'

data.loc[data['CTR'] > 1, 'CTR_Category'] = 'Invalid CTR' #There is one record with only 1 impression but 2 clicks. This is possible

print("CTR binned into 1% buckets.")


# 4. Number of Leads (Buckets of 5s)
# Bins: [0, 5), [5, 10), [10, 15), ... (Integers: 0-4, 5-9, 10-14, ...)
max_leads = data['Number_of_Leads'].max()
leads_bins = np.arange(0, max_leads + 5, 5)
leads_labels = [f'{int(leads_bins[i])}-{int(leads_bins[i+1])-1}' for i in range(len(leads_bins)-1)]
data['Leads_Category'] = pd.cut(data['Number_of_Leads'],
                                bins=leads_bins,
                                labels=leads_labels,
                                right=False,
                                include_lowest=True)
print("Number_of_Leads binned into buckets of 5.")


# 5. CPL (Buckets of $10)
# Bins: [0, 10), [10, 20), [20, 30), ...
cpl_nan_label = 'No_Leads'
max_cpl = data['CPL'].max()
cpl_bins = np.arange(0, max_cpl + 10, 10)
cpl_labels = [f'${int(cpl_bins[i])}-${int(cpl_bins[i+1])}' for i in range(len(cpl_bins)-1)]
data['CPL_Category'] = pd.cut(data['CPL'],
                              bins=cpl_bins,
                              labels=cpl_labels,
                              right=False,
                              include_lowest=True)
data['CPL_Category'] = data['CPL_Category'].cat.add_categories(['no leads'])
data.loc[data['Number_of_Leads'] == 0, 'CPL_Category'] = 'no leads'

print("CPL binned into $10 buckets.")

# --- Display Results ---
print("\nDataFrame with new category columns (first 5 rows):")
print(data[['CPM', 'CPM_Category', 'Amount_Spent', 'Amount_Spent_Category', 'CTR', 'CTR_Category', 'Number_of_Leads', 'Leads_Category', 'CPL', 'CPL_Category']].head())


Amount_Spent binned into 100s.
CTR binned into 1% buckets.
Number_of_Leads binned into buckets of 5.
CPL binned into $10 buckets.

DataFrame with new category columns (first 5 rows):
         CPM CPM_Category  Amount_Spent Amount_Spent_Category       CTR  \
0  17.028223   Quantile 1        900.91              900-1000  0.521670   
1  26.748129   Quantile 2       2564.00             2500-2600  0.375559   
2  27.017320   Quantile 2       2125.48             2100-2200  1.069009   
3  11.897764   Quantile 1        626.92               600-700  0.442192   
4  13.273481   Quantile 1        413.23               400-500  1.683156   

  CTR_Category  Number_of_Leads Leads_Category        CPL CPL_Category  
0        0%-1%               56          55-59  16.087679      $10-$20  
1        0%-1%               34          30-34  75.411765      $70-$80  
2  Invalid CTR               32          30-34  66.421250      $60-$70  
3        0%-1%               32          30-34  19.591250      $10-$20  
4

# HMM

Goal: Capture seasonality between months as a latent variable

Season_t   → Season_{t+1} \
Season_t   → Impressions_t \
Platform_t → Impressions_t \
Impressions_{t} → Impressions_{t+1} \
Month_t    → (observed input into Season_t or child of Season)

Model monthly impressions (counts) as arising from a latent seasonal regime that evolves over time via a Hidden Markov Model (HMM). Because impressions are non‐negative integer counts, a Poisson emission should capture their variance–mean relationship. We aggregate per‐platform monthly counts into separate time-series segments, fit a Poisson-HMM across all platforms (using sequence‐length information to respect platform boundaries), and select the number of latent seasons via the Bayesian Information Criterion (BIC). The resulting Viterbi states provide a point estimate of seasonality; the posterior probabilities give soft, probabilistic assignments to seasonal regimes.

In [463]:
hmm_df = data.copy()

In [427]:
# Converting month: E.g. 2023-03
hmm_df["Month"] = pd.to_datetime(hmm_df["Month"].str[:10]).dt.to_period("M")          
hmm_df.sort_values("Month", inplace=True)

# For each distinct (Month, Platform) pair, sum up all the raw Impressions - one time series per Platform
monthly_impressions = (hmm_df.groupby(["Month", "Platform"], as_index=False)["Impressions"].sum().sort_values(["Platform", "Month"]))

In [428]:
monthly_impressions

,Month,Platform,Impressions
0,2023-03,facebook,381652
2,2023-04,facebook,271476
4,2023-05,facebook,832828
6,2023-06,facebook,159864
8,2023-07,facebook,171866
10,2023-08,facebook,167444
12,2023-09,facebook,353770
14,2023-10,facebook,504318
16,2023-11,facebook,138501
18,2023-12,facebook,214031


In [301]:
# one per Platform, each shaped (T_i, 1) for that platform’s T_i months of data
seqs      = []
seq_lens  = []
for _ , g in monthly_impressions.groupby("Platform"):
    seqs.append(g["Impressions"].astype(int).values.reshape(-1, 1))
    seq_lens.append(len(g))
X  = np.concatenate(seqs, axis=0) # shape: (total_months_across_all_platforms, 1)
seq_lens = [len(s) for s in seqs]

In [302]:
# BIC to pick states
best_bic = np.inf  
results = []
k = 5 # Number of states to represent seasonality for each platform

model = hmm.PoissonHMM(n_components=k, n_iter=200, tol=1e-4, random_state=777)
model.fit(X, lengths=seq_lens)
ll = model.score(X, lengths=seq_lens)
bic = -2*ll + k * np.log(len(X))
if bic < best_bic:
    best_bic, best_model = bic, model

# Decode
post = best_model.predict_proba(X, lengths=seq_lens)
states = best_model.predict(X, lengths=seq_lens)

In [303]:
monthly_impressions["Seasonality"] = states
data = (hmm_df.merge(monthly_impressions[["Month","Platform","Seasonality"]], on=["Month","Platform"], how="left"))

## Improvements that can be made here

1. Impressions currently should not be distributed as a Poisson distribution. We should use a negative binomial distribution instead. This is because the variance of impressions is greater than the mean, which is a sign of overdispersion. The negative binomial distribution can account for this overdispersion.
2. We should also consider using a zero-inflated negative binomial distribution. This is because the data has a lot of zeros, which can be accounted for by the zero-inflated negative binomial distribution.

In [467]:
stats = (
    monthly_impressions
      .groupby("Platform")["Impressions"]
      .agg(mean="mean", var="var", n="size")
      .assign(var_mean_ratio=lambda d: d["var"] / d["mean"])
)

print(stats.round(2))

                mean           var   n  var_mean_ratio
Platform                                              
facebook   576796.76  1.931484e+11  25       334863.96
instagram  367149.52  8.498512e+10  25       231472.78


In [468]:
hmm_df = data.copy()
hmm_df["Month"] = pd.to_datetime(hmm_df["Month"].str[:10]).dt.to_period("M")
monthly_impressions = (hmm_df.groupby(["Month","Platform"], as_index=False)["Impressions"]
               .sum().sort_values(["Platform","Month"]))

In [469]:
seqs, seq_lens = [], []
for _, g in monthly_impressions.groupby("Platform"):
    seqs.append(torch.tensor(g["Impressions"].values.astype(float),
                             dtype=torch.float32))
    seq_lens.append(len(g))

device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X        = torch.cat(seqs).to(device)
seq_lens = torch.tensor(seq_lens, dtype=torch.long, device=device)

In [470]:
K = 5  # number of seasonal states

def model(data, lengths):
    # Initialization
    # init_logits spread across quantiles
    qs = np.quantile(X.cpu().numpy(), np.linspace(0.1,0.9,K))
    init_vals = torch.log(torch.tensor(qs, device=device) + 1e-3)
    init_noise= 0.1 * torch.randn(K, device=device)
    init_logits = pyro.param("init_logits",
                             init_vals + init_noise)

    # Sticky self-transitions: identity*K + jitter
    base_T = torch.eye(K, device=device) * 3.0    # 3 = self-bias
    jitter = 0.01 * torch.randn(K, K, device=device)
    trans_logits = pyro.param("trans_logits",
                              base_T + jitter)

    # Negative-Binom emission parameters seeded by data quantiles
    qs2       = np.percentile(X.cpu().numpy(), np.linspace(5,95,K))
    nb_logits = pyro.param("nb_logits",
                           torch.log(torch.tensor(qs2+1e-3,
                                                  device=device)))
    nb_r      = pyro.param("nb_r",
                           torch.linspace(1.0,5.0,K, device=device),
                           constraint=dist.constraints.positive)

    emission = dist.NegativeBinomial(total_count=nb_r,
                                     logits=nb_logits)

    hmm = DiscreteHMM(initial_logits   = init_logits,
                      transition_logits= trans_logits,
                      observation_dist = emission)

    # Accumulate per-platform log-likelihoods
    offset, total_lp = 0, 0.0
    for L in lengths.tolist():
        seq = data[offset:offset+L]
        total_lp = total_lp + hmm.log_prob(seq)
        offset += L

    pyro.factor("hmm_segments", total_lp)
    return total_lp

In [471]:
# SVI Fitting with random seed
def fit_hmm(seed):
    pyro.clear_param_store()          # wipe old pyro.param entries :contentReference[oaicite:5]{index=5}
    pyro.set_rng_seed(seed)           # reproducible :contentReference[oaicite:6]{index=6}

    guide = pyro.infer.autoguide.AutoDelta(model)
    svi   = SVI(model, guide, Adam({"lr":0.02}), loss=Trace_ELBO())

    for _ in range(500):
        svi.step(X, lengths=seq_lens)
    return svi.evaluate_loss(X, lengths=seq_lens)

best_nll, best_seed = float("inf"), None
seed = 85 # Already performed range(n) to find best seed
nll = fit_hmm(seed)

In [472]:
# Viterbi decoding
# retrieve final parameters
init_logits  = pyro.param("init_logits")
trans_logits = pyro.param("trans_logits")
nb_logits    = pyro.param("nb_logits")
nb_r         = pyro.param("nb_r")

log_init  = init_logits - torch.logsumexp(init_logits, dim=0)
log_trans = trans_logits - torch.logsumexp(trans_logits,
                                            dim=1, keepdim=True)
nb_dist   = torch.distributions.NegativeBinomial(total_count=nb_r,
                                                 logits=nb_logits)

def viterbi_path(obs):
    T = obs.size(0)
    B = nb_dist.log_prob(obs.unsqueeze(-1))  # (T, K)
    δ = torch.empty(T, K, device=device)
    ψ = torch.empty(T, K, dtype=torch.long, device=device)

    δ[0] = log_init + B[0]
    for t in range(1, T):
        scores = δ[t-1].unsqueeze(1) + log_trans
        δ[t], ψ[t] = scores.max(0)
        δ[t] += B[t]

    path = torch.empty(T, dtype=torch.long, device=device)
    path[-1] = δ[-1].argmax()
    for t in range(T-2, -1, -1):
        path[t] = ψ[t+1, path[t+1]]
    return path.cpu().numpy()

states, off = [], 0
for L in seq_lens.tolist():
    states.append(viterbi_path(X[off:off+L]))
    off += L

monthly_impressions["Seasonality"] = np.concatenate(states)
data = (hmm_df.merge(monthly_impressions[["Month","Platform","Seasonality"]],on=["Month","Platform"], how="left"))

# Sensortower Data Preprocessing

In [8]:
df_android = pd.read_csv('./Data/Sensor_Tower_App_Performance_Demographics_2024-10-01_to_2024-12-31_android.csv', sep='\t', encoding='utf-16')
df_ios = pd.read_csv('./Data/Sensor_Tower_App_Performance_Demographics_2024-10-01_to_2024-12-31_ios.csv', sep='\t', encoding='utf-16')
df_android['OS'] = 'android'
df_ios['OS'] = 'ios'
df_platform = pd.concat([df_android, df_ios], ignore_index=True)
df_platform = df_platform[~df_platform['App Name'].str.contains('Lite')]
cols = ['OS','App Name','Female 18-24','Female 25-34','Female 35-44','Female 45-54','Female 55-99','Male 18-24','Male 25-34','Male 35-44','Male 45-54','Male 55-99']
df_platform = df_platform[cols].rename(columns={'App Name': 'Platform'})

age_groups = ['18-24', '25-34', '35-44', '45-54', '55-99']
genders = ['Male', 'Female']
rows = []

# Iterate through each row in the original DataFrame
for _, row in df_platform.iterrows():
    for gender in genders:
        for age_group in age_groups:
            original_column = f'{gender} {age_group}'
            if original_column in df_platform.columns:
                probability = row[original_column]
                if not pd.isna(probability):
                    rows.append({
                        'OS': row['OS'],
                        'Platform': row['Platform'],
                        'Gender': gender,
                        'Age_Group': age_group,
                        'Probability': float(probability.strip('%')) / 100
                    })
                    
df_platform = pd.DataFrame(rows)
df_platform

,OS,Platform,Gender,Age_Group,Probability
0,android,Facebook,Male,18-24,0.1735
1,android,Facebook,Male,25-34,0.2345
2,android,Facebook,Male,35-44,0.1303
3,android,Facebook,Male,45-54,0.0549
4,android,Facebook,Male,55-99,0.0141
5,android,Facebook,Female,18-24,0.1146
6,android,Facebook,Female,25-34,0.1509
7,android,Facebook,Female,35-44,0.0823
8,android,Facebook,Female,45-54,0.0356
9,android,Facebook,Female,55-99,0.0093


# DAG

In [ ]:
amt_spent = 'Amount_Spent_Category'
cpm = 'CPM_Category'
ctr = 'CTR_Category'
leads = 'Leads_Category' # Corresponds to N
cpl = 'CPL_Category'     # Corresponds to L
device = 'Device_platform' # Corresponds to DP
platform = 'Platform'     # Corresponds to PL
custom_aud = 'Use_Custom_Audience' # Corresponds to CA
exclusions = 'Use_Exclusions'     # Corresponds to EX
objective = 'Campaign_Objective' # Corresponds to CO
hl = 'Headline_Local'
he = 'Headline_Event'
hx = 'Headline_Exclusivity'
hr = 'Headline_Rental'
hrt = 'Headline_Returns' # Assuming maps to HRTgit c
hd = 'Headline_Discounts'
reach = 'Reach'
frequency = 'Frequency'
impressions = 'Impressions'
gender = 'Gender'
age = 'Age'

# List of all nodes expected in the model_data DataFrame
all_nodes = [
    amt_spent, cpm, ctr, leads, cpl, device, platform, custom_aud,
    exclusions, objective, hl, he, hx, hr, hrt, hd, reach, frequency, impressions
]


# Elements that can be controlled
headlines = [hl, he, hx, hr, hrt, hd]
settings_context_nodes = [amt_spent, device, platform, custom_aud, exclusions, objective]
all_settings_nodes = list(set(settings_context_nodes + headlines)) # Use set to ensure unique

In [10]:
model = DiscreteBayesianNetwork()

print(f"Checking/Adding nodes: {all_nodes}")
# Optional: Check against data columns first
missing_nodes = [node for node in all_nodes if node not in data.columns]
if missing_nodes:
    print(f"ERROR: Nodes missing from data columns: {missing_nodes}")
    present_nodes = [node for node in all_nodes if node in data.columns]
    print(f"Warning: Adding only nodes present in data: {present_nodes}")
    model.add_nodes_from(present_nodes)
else:
    print("All nodes found. Adding all nodes.")
    model.add_nodes_from(all_nodes) # Add all defined nodes to the model object

# Define Edges (Parent -> Child relationships) based on plausible influence flow
# Using a slightly simplified structure to reduce excessive parent numbers
edges = []
present_model_nodes = list(model.nodes()) # Work only with nodes actually added

# Function to safely add edge if both nodes exist in the model
def add_safe_edge(parent, child):
    if parent in present_model_nodes and child in present_model_nodes and parent != child:
        edges.append((parent, child))

# Settings -> CPM (M)
parents_for_cpm = settings_context_nodes
for parent in parents_for_cpm:
    add_safe_edge(parent, cpm)

# Settings + Headlines + CPM -> CTR (T)
parents_for_ctr = settings_context_nodes + headlines + [cpm]
for parent in parents_for_ctr:
    add_safe_edge(parent, ctr)

# Settings + Headlines + CTR -> Leads (N)
parents_for_leads = settings_context_nodes + headlines + [ctr]
for parent in parents_for_leads:
    add_safe_edge(parent, leads)

# Settings + Metrics (CPM, CTR) + Leads -> CPL (L)
parents_for_cpl = settings_context_nodes + [cpm, ctr, leads]
for parent in parents_for_cpl:
    add_safe_edge(parent, cpl)

# Add all defined edges to the model
model.add_edges_from(edges)

Checking/Adding nodes: ['Amount_Spent_Category', 'CPM_Category', 'CTR_Category', 'Leads_Category', 'CPL_Category', 'Device_platform', 'Platform', 'Use_Custom_Audience', 'Use_Exclusions', 'Campaign_Objective', 'Headline_Local', 'Headline_Event', 'Headline_Exclusivity', 'Headline_Rental', 'Headline_Returns', 'Headline_Discounts', 'Reach', 'Frequency', 'Impressions']
All nodes found. Adding all nodes.
